In [1]:
!pip install tflearn

In [2]:
#we used tensorboard to visualize the accuracy and loss graphs

In [3]:
rm -rf ./logs/

In [4]:
%load_ext tensorboard 

In [5]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tensorflow as tf
import tflearn
import random
import json
import pickle
import datetime, os

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=1)

import tensorflow as tflow

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Instructions for updating:
non-resource variables are not supported in the long term


# **DATA PRE-PROCESSING**
pattern = questions

tag = type of question

response = answer of pattern

In [6]:
stemmer = LancasterStemmer()

# accessing the json file that has the questions and answers
with open("dataset.json") as file:
    data = json.load(file)

#print(data)
#print(data["intents"])

In [7]:
try:
    x    # uncomment the 'x' after changing the JSON file so that it reprocesses the new changes
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)  # loads variables into pickle file, and then reuses them

except:
    words = []  # tokenized words of the pattern
    labels = [] # tag labels we used
    docs_x = [] # for each word in pattern/question in docs_x we have
    docs_y = [] # the intent tag it has in docs_y

    # tokinizing the words of the statements
    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)        
            docs_y.append(intent["tag"])  

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    # stemming the questions into base words
    words = [stemmer.stem(w.lower()) for w in words if w != "?"]  #takes the pattern words and sets them to small caps for equal consideration
    words = sorted(list(set(words)))                              # set removes duplicates, then return into sorted words

    labels = sorted(labels)

    # creating bag of words: one hot encoding
    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for question, doc in enumerate(docs_x):
        bag = []
        wrds = [stemmer.stem(w) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)   # holds 1 if the word exists in pattern
            else:
                bag.append(0)   # holds 0 if the word is not in pattern
        
        output_row = out_empty[:]
        output_row[labels.index(docs_y[question])] = 1     # we look through our used tags and set it to 1 in ouput_row
        training.append(bag)                               # training data
        output.append(output_row)                          # output data

    # turning them into numpy arrays for model use
    training = np.array(training)
    output = np.array(output)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)  # saves variables into pickle file, and then reuses them



# MODEL TRAINING

In [8]:
# tensorflow and tflearn model
net = tflearn.input_data(shape = [None, len(training[0])])

# add more neurons and/or layers for more data and/or more tags
net = tflearn.fully_connected(net, 600, activation = 'relu')   # connect first hidden layer of network with 8 nodes
net = tflearn.fully_connected(net, 1222, activation = 'relu')   # second hidden layer
net = tflearn.fully_connected(net, 1200, activation = 'relu')   # third hidden layer
net = tflearn.fully_connected(net, 1000, activation = 'relu')   # fifth hidden layer
net = tflearn.fully_connected(net, 500, activation = 'relu')   # fifth hidden layer
net = tflearn.fully_connected(net, 200, activation = 'relu')   # fifth hidden layer

# output layer: softmax allows us to get probability for each output tag according to the input
net = tflearn.fully_connected(net, len(output[0]), activation = "softmax") 

net = tflearn.regression(net, optimizer = 'adam', loss = 'categorical_crossentropy')

model = tflearn.DNN(net, checkpoint_path='/tmp/tflearn_logs/', tensorboard_verbose=0)



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
model.fit(training, output, n_epoch = 310, batch_size = 150,show_metric = True,run_id='Chatbot')
model.save("model.tflearn")

Training Step: 929  | total loss: 1.47647 | time: 0.150s
| Adam | epoch: 310 | loss: 1.47647 - acc: 0.9062 -- iter: 300/303
Training Step: 930  | total loss: 1.33751 | time: 0.260s
| Adam | epoch: 310 | loss: 1.33751 - acc: 0.9109 -- iter: 303/303
--
INFO:tensorflow:/tmp/tflearn_logs/-930 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [10]:
!tensorboard dev upload \
  --logdir '/tmp/tflearn_logs/Chatbot' \
  --name "Chatbot" \
  --one_shot

2021-12-11 21:31:40.081352: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected

New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/0IYJzeoKSJWwfaNKboBYOw/

[2021-12-11T21:31:40] Started scanning logdir.
E1211 21:31:41.643534 139670005086080 uploader.py:1122] Attempted to re-upload existing blob.  Skipping.
E1211 21:31:42.624364 139670005086080 uploader.py:1122] Attempted to re-upload existing blob.  Skipping.
[2021-12-11T21:31:43] Total uploaded: 3948 scalars, 0 tensors, 1 binary objects (151.4 kB)
Total skipped: 2 binary objects (302.8 kB)
[2021-12-11T21:31:43] Done scanning logdir.


Done. View your TensorBoard at https://tensorboard.dev/experiment/0IYJzeoKSJWwfaNKboBYOw/


In [12]:
# function that bags the user input words, and changes them into numpy arrays for use for model
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return np.array(bag)

# function to start chatting with the bot/model
def chat():
    print("Start talking with AUBot (type quit to stop)!")

    while True:
        inp = input("\nYou: ")
        if inp.lower() == "quit":
            break

        print("\nAUBot: ")

        results = model.predict([bag_of_words(inp, words)])  # classifies the input into a probability array of what tag it should be
        #print(results)
        results_index = np.argmax(results)  # index of highest probabile tag
        tag = labels[results_index]
        #print(tag)

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

chat()


Start talking with AUBot (type quit to stop)!

You: what's up

AUBot: 
Hello! how are doing today?

You: how are you doing

AUBot: 
I am doing just fine. Please ask me some question.

You: what's your name

AUBot: 
You can call me AUBot.

You: what can I ask you

AUBot: 
i can answer AUB related questions relavant to transcripts, financial aid, capacities, petitions, registration, transfer, and work study.

You: what does a negative value mean on the web statement of fees

AUBot: 
It is for financial aid, what you have removed. The statement will update later.

You: does financial aid cover the summer semester

AUBot: 
If you ask nicely maybe 

You: when does the registrar open

AUBot: 
Registrar, Cashier and Comptroller's offices are from 8 a.m. to 4 p.m. M-F outside of holidays.

You: how do I submit a petition

AUBot: 
go to the following link: https://epetitions.aub.edu.lb/.

You: my web statement of fees is not updated even though I paid

AUBot: 
Check with the Comptroller's at AU